# Assessment Problems

Python project that downloads FAANG stock data, plots it, automates execution via a script, and schedules it weekly with GitHub Actions.

## Problem 1: Data from yFinance

Function `get_data()` uses [yfinance](https://github.com/ranaroussi/yfinance) Python package to:

1. Download hourly stock data for the past 5 days for FAANG (META, AAPL, AMZN, NFLX, GOOG).

2. Save results as a CSV in a data/ folder (create it if missing).

The filename format is `YYYYMMDD-HHmmss.csv` (date + time of save).

In [1]:
# Dates and times
import datetime as dt

# Data frames
import pandas as pd

#  Yahoo Finance yfinance module to fetch stock data
import yfinance as yf

Creating Tickers object for FAANG stocks
Tickers  
https://ranaroussi.github.io/yfinance#ticker


In [2]:
# Downloading stock data for Apple, Microsoft, Google, Amazon, and Tesla 
ticker_symbols = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"]
# Fetching stock data using yfinance. It returns a Tickers dataframe
stock_data = yf.download(ticker_symbols, period="5d", interval="1h")

/var/folders/71/lgn75sbs3sngtkrhdkwngslm0000gn/T/ipykernel_24784/1717889272.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(ticker_symbols, period="5d", interval="1h")
[*********************100%***********************]  5 of 5 completed


Date and time formatting for filename  
https://docs.python.org/3/library/datetime.html

In [3]:
# Getting current date and time for filename with format YYYYMMDD-HHmmss
# https://docs.python.org/3/library/datetime.html#datetime.datetime.now
current_time = dt.datetime.now().strftime("%Y%m%d-%H%M%S")

# Creating full file path (including the filename) for saving the stock data
full_filepath = f"data/{current_time}.csv"

Saving the fetched stock data to a CSV file  
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html

In [4]:
# Saving the CSV file in data folder with name as per the specified format
stock_data.to_csv(full_filepath)
# Printing the fetched stock data
print(stock_data)

Price                           Close                                      \
Ticker                           AAPL        AMZN       GOOGL        MSFT   
Datetime                                                                    
2025-10-21 13:30:00+00:00  263.499908  221.720001  251.815002  514.854980   
2025-10-21 14:30:00+00:00  263.661987  221.970001  246.229996  516.049988   
2025-10-21 15:30:00+00:00  263.170013  222.975006  248.934998  515.729980   
2025-10-21 16:30:00+00:00  263.326508  221.544998  251.600006  514.210022   
2025-10-21 17:30:00+00:00  263.449890  222.584305  252.020004  516.015015   
2025-10-21 18:30:00+00:00  263.084991  222.460007  251.460007  517.195007   
2025-10-21 19:30:00+00:00  262.839996  222.029999  250.429993  517.739990   
2025-10-22 13:30:00+00:00  260.049896  217.449997  252.869995  523.465027   
2025-10-22 14:30:00+00:00  258.429901  218.739899  252.570007  522.710022   
2025-10-22 15:30:00+00:00  258.549988  218.970001  251.255005  523.005005   

## Problem 2: Plotting Data

Function `plot_data()` loads the latest CSV in the `data` folder, plots `Close` prices for all five FAANG stocks with labels, legend, and date as the title.
Then it saves the plot in a `plots` folder as `YYYYMMDD-HHmmss.png` by creating the folder if it doesn't exist.

In [5]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

def plot_data():
    # Find the latest CSV in data/ using glob
    # https://docs.python.org/3/library/glob.html
    csv_files = glob.glob('data/*.csv')
    if not csv_files:
        # If no csv file exists, return
        print("No CSV files found in data/")
        return
    # Get the latest CSV file using os.path.getctime
    # https://www.geeksforgeeks.org/python/python-os-path-getctime-method/
    latest_csv = max(csv_files, key=os.path.getctime)

    # Load CSV with multi-index columns
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
    df = pd.read_csv(latest_csv, header=[0,1], index_col=0, parse_dates=True)

    # Plot Close prices for all available tickers
    plt.figure(figsize=(12,6))
    
    # Get all tickers in the second level of the multi-index columns
    # We need it for ticker in df.columns.levels[1]: because our CSV uses a MultiIndex for columns:
    # 1. The first level is the price type (Close, High, etc.)
    # 2. The second level is the ticker symbol (AAPL, AMZN, etc.)
    # df.columns.levels[1] gives all unique ticker symbols. So we can loop for each ticker symbol
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.MultiIndex.levels.html
    # I also had some help from ChatGPT to get tickers under the "Close" price type
    # https://chatgpt.com/share/68ffd5b2-6fe4-800f-82bb-25299b654018
    tickers_close = [ticker for price_type, ticker in df.columns if price_type == "Close"]
    # Loop through each ticker and plot its Close price
    for ticker in tickers_close:
        # Plot Close price for each ticker
        # https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html
        plt.plot(df.index, df[("Close", ticker)], label=ticker)
    # Add axis labels, legend, and title
    plt.xlabel("Time")
    plt.ylabel("Close Price")
    plt.legend()
    plt.title(f"FAANG Close Prices - {os.path.basename(latest_csv).split(".")[0]}")

    # Save plot
    os.makedirs("plots", exist_ok=True)
    # Format the filename with date and time
    # https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
    out_path = f"plots/{dt.datetime.now().strftime('%Y%m%d-%H%M%S')}.png"
    # Save plot
    # https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.savefig.html
    plt.savefig(out_path)
    # Close plot to free up memory
    # https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.close.html
    plt.close()

In [6]:
# Calling the function to generate the plot in the plots folder
plot_data()

## Problem 3: Script

Script `faang.py` in the repo root contains the two functions (`get_data()` and `plot_data()`). It can be run by `./faang.py` from the terminal to download the data and generate the plot.

## Problem 4: Automation

GitHub Actions workflow (`faang.yml`) inside `.github/workflows/` runs `faang.py` every Saturday morning.

## End